In [1]:
import os
import numpy as np
import pandas as pd 
from tqdm import tqdm_notebook
import matplotlib.pyplot as plt
%matplotlib inline

import warnings
warnings.filterwarnings('ignore')

import seaborn as sns
import datetime
import pickle

import nltk
from string import punctuation
from collections import OrderedDict
import matplotlib.pyplot as plt
from operator import itemgetter

In [2]:
df = pd.read_pickle('final.p')

In [3]:
df.iloc[1]

Título                                    Ótimo carro a um custo baixo!
Carro                                  Chevrolet Celta LT 1.0 2015/2015
Dono                                                               Hugo
Prós                  Motor potente pra um carro 1.0, sem deixar de ...
Contras                                                   Saiu de linha
Defeitos                                                         Nenhum
Opinião               Este é o segundo celta que compro 0km e não me...
Data                                                1520026020000000000
Estilo                                                               10
Acabamento                                                            8
Posição de dirigir                                                    8
Instrumentos                                                          8
Interior                                                              6
Porta-malas                                                     

In [4]:
df = df[['Marca', 'Prós', 'Contras', 'Defeitos', 'Opinião']]

In [5]:
with open('./stopwords.txt','r', encoding='utf-8') as f:
    stop_words = f.read()

def clean_data(x):

   x =  ''.join([c for c in x if c not in punctuation])
   x = x.lower()
   x = x

   return x

In [6]:
    
df['Prós'] = df['Prós'].apply(lambda x: clean_data(x))
df['Contras'] = df['Contras'].apply(lambda x: clean_data(x))
df['Defeitos'] = df['Defeitos'].apply(lambda x: clean_data(x))
df['Opinião'] = df['Opinião'].apply(lambda x: clean_data(x))

In [7]:
def limit_set(aux,min_length,min_freq):
    '''
    Metodo para limitar o bag of Words, existem palavras com poucas frequencias
    e palavras nao muito uteis como artigos (e,as ,os), existem muitas palavras que são numeros
    relacionados a quilometragem, que contribuem para o ruído do dataset
    '''
    keys = []
    for key in aux.keys():
        if len(key) < min_length or aux[key] < min_freq or key.isdigit() or key in stop_words or key.find('km')!=-1 or key.find('mil')!=-1:
            keys.append(key)
            
    for key in keys:
        del aux[key]
        
    return aux

In [8]:
df.head()

,Marca,Prós,Contras,Defeitos,Opinião
ID,,,,,
e2b9dc08,Chevrolet,economia de combustível peças baratas resisten...,um pouco baixo as vezes quando está carregado ...,nenhum,o carro é 10 estou com ele a 7 anos e rodei 35...
3b9dcf63,Chevrolet,motor potente pra um carro 10 sem deixar de la...,saiu de linha,nenhum,este é o segundo celta que compro 0km e não me...
9f62a709,Chevrolet,tamanhocabe em qualquer vagaexcelente para and...,nao tem levando em conta que o carro foi feito...,até agora nenhum já fiz algumas viagens e graç...,recomendo carro valente relativamente confortá...
0e6c8d29,Chevrolet,carro extremamente confiavel so fazer a manute...,acabamento do carro é rustico bem mal feito ba...,nada o carro é valente,ja passei ate por trilhas onde caminhonetes fi...
3c958631,Chevrolet,desempenhopara 10 manutenção barata carro bem ...,meio duro pouco espaço motor grita muito em vi...,bomba dágua,carro honesto para primeiro veiculo melhor que...


In [9]:
def clean_stopwords(x):
    word_list = x.split(' ')
    return [word for word in word_list if word not in stop_words]

In [10]:
def df_clean_stopwords(df):
    df['Prós'] = df['Prós'].apply(lambda x: clean_stopwords(x))
    df['Contras'] = df['Contras'].apply(lambda x: clean_stopwords(x))
    df['Defeitos'] = df['Defeitos'].apply(lambda x: clean_stopwords(x))
    df['Opinião'] = df['Opinião'].apply(lambda x: clean_stopwords(x))

In [11]:
df_clean_stopwords(df)

In [12]:
df.head()

,Marca,Prós,Contras,Defeitos,Opinião
ID,,,,,
e2b9dc08,Chevrolet,"[economia, combustível, peças, baratas, resist...","[pouco, baixo, vezes, carregado, pega, baixo, ...",[nenhum],"[carro, 10, 7, anos, rodei, 35000, kmrodo, pou..."
3b9dcf63,Chevrolet,"[motor, potente, pra, carro, 10, deixar, lado,...","[saiu, linha]",[nenhum],"[segundo, celta, compro, 0km, arrependo, faço,..."
9f62a709,Chevrolet,"[tamanhocabe, qualquer, vagaexcelente, andar, ...","[nao, levando, conta, carro, feito, pensando, ...","[agora, nenhum, fiz, algumas, viagens, graças,...","[recomendo, carro, valente, relativamente, con..."
0e6c8d29,Chevrolet,"[carro, extremamente, confiavel, fazer, manute...","[acabamento, carro, rustico, bem, mal, feito, ...","[nada, carro, valente]","[passei, ate, trilhas, onde, caminhonetes, fic..."
3c958631,Chevrolet,"[desempenhopara, 10, manutenção, barata, carro...","[meio, duro, pouco, espaço, motor, grita, viag...","[bomba, dágua]","[carro, honesto, primeiro, veiculo, melhor, go..."


In [13]:
def df_get_bigrams(df):
    df['bi_Prós'] = df['Prós'].apply(lambda x: list(nltk.ngrams(x ,2)))
    df['bi_Contras'] = df['Contras'].apply(lambda x: list(nltk.ngrams(x ,2)))
    df['bi_Defeitos'] = df['Defeitos'].apply(lambda x: list(nltk.ngrams(x ,2)))
    df['bi_Opinião'] = df['Opinião'].apply(lambda x: list(nltk.ngrams(x ,2)))

In [14]:
df_get_bigrams(df)

In [15]:
def df_get_trigrams(df):
    df['tri_Prós'] = df['Prós'].apply(lambda x: list(nltk.ngrams(x, 3)))
    df['tri_Contras'] = df['Contras'].apply(lambda x: list(nltk.ngrams(x, 3)))
    df['tri_Defeitos'] = df['Defeitos'].apply(lambda x: list(nltk.ngrams(x, 3)))
    df['tri_Opinião'] = df['Opinião'].apply(lambda x: list(nltk.ngrams(x, 3)))

In [16]:
df_get_trigrams(df)

In [17]:
df.iloc[1]

Marca                                                   Chevrolet
Prós            [motor, potente, pra, carro, 10, deixar, lado,...
Contras                                             [saiu, linha]
Defeitos                                                 [nenhum]
Opinião         [segundo, celta, compro, 0km, arrependo, faço,...
bi_Prós         [(motor, potente), (potente, pra), (pra, carro...
bi_Contras                                        [(saiu, linha)]
bi_Defeitos                                                    []
bi_Opinião      [(segundo, celta), (celta, compro), (compro, 0...
tri_Prós        [(motor, potente, pra), (potente, pra, carro),...
tri_Contras                                                    []
tri_Defeitos                                                   []
tri_Opinião     [(segundo, celta, compro), (celta, compro, 0km...
Name: 3b9dcf63, dtype: object

In [18]:
df_renault = df[df['Marca'] == 'Renault']
df_chevrolet = df[df['Marca'] == 'Chevrolet']
df_fiat = df[df['Marca'] == 'Fiat']
df_hyundai = df[df['Marca'] == 'Hyundai']
df_volkswagen = df[df['Marca'] == 'Volkswagen']

In [19]:
df_list = [df_renault, df_chevrolet, df_fiat, df_hyundai, df_volkswagen]

In [20]:
listas_bi_pros = []
for i, df in enumerate(df_list):
    bi_pros_list = []
    for index, row in df.iterrows():
        bi_pros_list += row['bi_Prós']
    listas_bi_pros.append(bi_pros_list)

In [21]:
listas_bi_contras = []
for i, df in enumerate(df_list):
    bi_pros_list = []
    for index, row in df.iterrows():
        bi_pros_list += row['bi_Contras']
    listas_bi_contras.append(bi_pros_list)

In [22]:
listas_bi_defeitos = []
for i, df in enumerate(df_list):
    bi_pros_list = []
    for index, row in df.iterrows():
        bi_pros_list += row['bi_Defeitos']
    listas_bi_defeitos.append(bi_pros_list)

In [23]:
listas_tri_pros = []
for i, df in enumerate(df_list):
    tri_pros_list = []
    for index, row in df.iterrows():
        tri_pros_list += row['tri_Prós']
    listas_tri_pros.append(tri_pros_list)

In [24]:
listas_tri_contras = []
for i, df in enumerate(df_list):
    tri_pros_list = []
    for index, row in df.iterrows():
        tri_pros_list += row['tri_Contras']
    listas_tri_contras.append(tri_pros_list)

In [25]:
listas_tri_defeitos = []
for i, df in enumerate(df_list):
    tri_pros_list = []
    for index, row in df.iterrows():
        tri_pros_list += row['tri_Defeitos']
    listas_tri_defeitos.append(tri_pros_list)

# <br><br><font color='#cccc00'>Renault</font>

## Bigramas Prós
Espaço Interno,
Porta Malas,
Posição Dirigir

In [43]:
fdist = nltk.FreqDist(listas_bi_pros[0])
for k,v in fdist.items():
    if v > 25:
        print (k,v)

('bom', 'espaço') 47
('espaço', 'interno') 300
('interno', 'porta') 34
('porta', 'malas') 136
('carro', 'bonito') 27
('computador', 'bordo') 27
('posição', 'dirigir') 88
('interno', 'bom') 33
('3', 'anos') 31
('motor', '16') 29
('ótimo', 'espaço') 26


## Bigramas Contras
Vidros Elétricos, Barulho Motor, Isolamento Acústico, Acabamento Interno

In [37]:
fdist = nltk.FreqDist(listas_bi_contras[0])
for k,v in fdist.items():
    if v > 20:
        print (k,v)

('acabamento', 'interno') 28
('motor', '10') 21
('poderia', 'melhor') 22
('barulho', 'motor') 29
('vidros', 'elétricos') 33
('isolamento', 'acústico') 28


## Bigramas Defeitos
Mil km, Nenhum Defeito, Nenhum Momento

In [95]:
fdist = nltk.FreqDist(listas_bi_defeitos[0])
for k,v in fdist.items():
    if v > 10:
        print (k,v)

('mil', 'km') 37
('km', 'rodados') 20
('caixa', 'direção') 15
('30000', 'km') 11
('correia', 'dentada') 11
('banco', 'motorista') 17
('agora', 'nenhum') 13
('nenhum', 'agora') 12
('apresentou', 'defeito') 16
('porta', 'motorista') 13
('nenhum', 'momento') 26
('apresentou', 'nenhum') 11
('nenhum', 'defeito') 37
('km', 'apresentou') 12
('10000', 'km') 11
('momento', 'nenhum') 14


## Trigramas Prós
Bom espaco interno, Interno porta malas, Boa posicao para dirigir

In [97]:
fdist = nltk.FreqDist(listas_tri_pros[0])
for k,v in fdist.items():
    if v > 15:
        print (k,v)

('bom', 'espaço', 'interno') 31
('espaço', 'interno', 'porta') 32
('interno', 'porta', 'malas') 32
('excelente', 'espaço', 'interno') 18
('boa', 'posição', 'dirigir') 18
('espaço', 'interno', 'bom') 30
('ótimo', 'espaço', 'interno') 23
('espaço', 'interno', 'portamalas') 18
('espaço', 'interno', 'excelente') 25


## Trigramas Contras
Botões vidros eletricos, Faz (um) pouco barulho, Estepe embaixo carro

In [49]:
fdist = nltk.FreqDist(listas_tri_contras[0])
for k,v in fdist.items():
    if v > 4:
        print (k,v)

('estepe', 'baixo', 'carro') 5
('botões', 'vidros', 'elétricos') 7
('deixa', 'pouco', 'desejar') 6
('estepe', 'embaixo', 'carro') 8
('faz', 'pouco', 'barulho') 5


## Trigramas Defeitos
Defeito em 20 mil km, Apresentou nenhum defeito, Mil km rodados

In [53]:
fdist = nltk.FreqDist(listas_tri_defeitos[0])
for k,v in fdist.items():
    if v > 3:
        print (k,v)

('mil', 'km', 'rodados') 6
('40', 'mil', 'km') 4
('apresentou', 'nenhum', 'defeito') 8
('nenhum', 'ate', 'agora') 4
('nenhum', 'defeito', 'apresentado') 8
('revisão', '20', 'mil') 4
('barulho', 'roda', 'dianteira') 4
('km', 'rodados', 'nenhum') 4
('km', 'apresentou', 'defeito') 4


# <br><br><font color='#3399ff'>Volkswagen</font>

## Bigramas Prós
Espaço Interno, Posição dirigir, Carro Bonito

In [56]:
fdist = nltk.FreqDist(listas_bi_pros[4])
for k,v in fdist.items():
    if v > 60:
        print (k,v)

('posição', 'dirigir') 157
('manutenção', 'barata') 103
('motor', '16') 63
('computador', 'bordo') 69
('carro', 'bonito') 109
('carro', 'bom') 68
('espaço', 'interno') 242
('anda', 'bem') 79


## Bigramas Contras
Porta Malas, Suspensão Dura, Malas Pequeno

In [59]:
fdist = nltk.FreqDist(listas_bi_contras[4])
for k,v in fdist.items():
    if v > 50:
        print (k,v)

('porta', 'malas') 229
('malas', 'pequeno') 89
('suspensão', 'dura') 106
('acabamento', 'interno') 84
('seguro', 'caro') 53
('faz', 'barulho') 58
('posição', 'dirigir') 53


## Bigramas Defeitos

In [63]:
fdist = nltk.FreqDist(listas_bi_defeitos[4])
for k,v in fdist.items():
    if v > 35:
        print (k,v)

('vidros', 'elétricos') 37
('duas', 'vezes') 43
('nenhum', 'defeito') 52
('mil', 'km') 90
('vidro', 'elétrico') 40
('1', 'ano') 37
('km', 'rodados') 55
('correia', 'dentada') 36


## Trigramas Prós

In [66]:
fdist = nltk.FreqDist(listas_tri_pros[4])
for k,v in fdist.items():
    if v > 15:
        print (k,v)

('espaço', 'interno', 'bom') 26
('bom', 'espaço', 'interno') 36
('nunca', 'deixou', 'mão') 20
('boa', 'posição', 'dirigir') 23
('baixo', 'custo', 'manutenção') 23


## Trigramas Contras
Porta Malas Pequeno, Suspensão Pouco Dura, Alto Consumo Combustível


In [70]:
fdist = nltk.FreqDist(listas_tri_contras[4])
for k,v in fdist.items():
    if v > 7:
        print (k,v)

('porta', 'malas', 'pequeno') 85
('porta', 'mala', 'pequeno') 12
('suspensão', 'pouco', 'dura') 17
('cada', '6', 'meses') 11
('alto', 'consumo', 'combustível') 10
('ponto', 'cego', 'coluna') 11
('parece', 'carro', 'vai') 8
('altura', 'banco', 'motorista') 11
('parece', 'escola', 'samba') 10
('motor', '10', 'fraco') 14
('motor', 'faz', 'barulho') 8
('muitos', 'ruídos', 'internos') 9
('regulagem', 'altura', 'banco') 9


## Trigramas Defeitos
Apresentou Nenhum Defeito, 2 Ano Uso, Mil Km Rodados

In [98]:
fdist = nltk.FreqDist(listas_tri_defeitos[4])
for k,v in fdist.items():
    if v > 7:
        print (k,v)

('velas', 'cabos', 'velas') 9
('cabos', 'velas', 'velas') 8
('apresentou', 'nenhum', 'defeito') 15
('40', 'mil', 'km') 8
('1', 'ano', 'uso') 8
('2', 'anos', 'uso') 13
('ate', 'agora', 'nenhum') 8
('mil', 'km', 'rodados') 14


# <br><br><font color='#999966'>Hyundai</font>

## Bigramas Prós
Carro Bonito, Porta Malas, Espaço Interno

In [99]:
fdist = nltk.FreqDist(listas_bi_pros[3])
for k,v in fdist.items():
    if v > 23:
        print (k,v)

('porta', 'malas') 37
('5', 'anos') 27
('carro', 'bonito') 51
('acabamento', 'interno') 25
('espaço', 'interno') 30


## Bigramas Contras
Suspensão Traseira, Espaço Interno, Porta Malas 

In [100]:
fdist = nltk.FreqDist(listas_bi_contras[3])
for k,v in fdist.items():
    if v > 18:
        print (k,v)

('banco', 'motorista') 19
('porta', 'malas') 31
('espaço', 'interno') 32
('suspensão', 'traseira') 44
('banco', 'traseiro') 26
('deixa', 'desejar') 19
('computador', 'bordo') 20


## Bigramas Defeitos

In [94]:
fdist = nltk.FreqDist(listas_bi_defeitos[3])
for k,v in fdist.items():
    if v > 10:
        print (k,v)

('nenhum', 'momento') 33
('mil', 'km') 11
('nenhum', 'defeito') 27
('momento', 'nenhum') 24


## Trigramas Prós
Garantia 5 anos, Bonito chama atenção, Leve parece Elétrico

In [92]:
fdist = nltk.FreqDist(listas_tri_pros[3])
for k,v in fdist.items():
    if v > 4:
        print (k,v)

('condicionado', 'gela', 'bem') 5
('garantia', '5', 'anos') 15
('5', 'anos', 'garantia') 10
('carro', 'bonito', 'chama') 5
('bonito', 'chama', 'atenção') 7
('leve', 'parece', 'elétrica') 5
('acabamento', 'interno', 'bom') 5


## Trigramas Contras
Rugulagem Altura Banco, Altura Banco Motorista, Suspensão (um) pouco dura

In [89]:
fdist = nltk.FreqDist(listas_tri_contras[3])
for k,v in fdist.items():
    if v > 4:
        print (k,v)

('suspensão', 'pouco', 'dura') 7
('espaço', 'interno', 'traseiro') 6
('regulagem', 'altura', 'banco') 10
('altura', 'banco', 'motorista') 10
('luz', 'porta', 'malas') 7
('luz', 'porta', 'luvas') 5
('duas', 'pessoas', 'banco') 6
('computador', 'bordo', 'poderia') 5


## Trigramas Defeitos
Nenhum Defeito Momento, Nenhum Defeito Apresentado, Apresentou nenhum defeito

In [90]:
fdist = nltk.FreqDist(listas_tri_defeitos[3])
for k,v in fdist.items():
    if v > 3:
        print (k,v)

('apresentou', 'nenhum', 'defeito') 5
('nenhum', 'defeito', 'momento') 6
('nenhum', 'defeito', 'apresentado') 5
('km', 'rodados', 'nenhum') 4
